In [ ]:
# Install the wheel files
%pip install /lakehouse/default/Files/unified_etl_core-1.0.0-py3-none-any.whl
%pip install /lakehouse/default/Files/unified_etl_connectwise-1.0.0-py3-none-any.whl

In [2]:
# Alternative: Automatically discover table mappings
def discover_table_mappings(spark, prefix_pattern="bronze_cw_"):
    """Discover existing tables and create mappings."""
    tables = spark.sql(f"SHOW TABLES LIKE '{prefix_pattern}*'").collect()

    mappings = {"bronze": {}, "silver": {}, "gold": {}}

    for row in tables:
        table_name = row.tableName
        # Extract entity name from table name
        if table_name.startswith(prefix_pattern):
            entity_name = table_name.replace(prefix_pattern, "")
            mappings["bronze"][entity_name] = table_name
            # Assume silver and gold follow same pattern
            mappings["silver"][entity_name] = f"silver_cw_{entity_name}"
            mappings["gold"][entity_name] = f"gold_cw_{entity_name}"

    return mappings


# Discover existing tables
discovered_mappings = discover_table_mappings(spark)
print("Discovered table mappings:")
for layer, entities in discovered_mappings.items():
    if entities:
        print(f"\n{layer.upper()}:")
        for entity, table in entities.items():
            print(f"  {entity} -> {table}")

StatementMeta(, 89f93da4-344b-49ff-8a91-0fe14404ed3d, 11, Finished, Available, Finished)

Discovered table mappings:


In [6]:
from unified_etl_core.main import run_etl_pipeline
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

print("🔍 Discovering tables in schema-enabled lakehouse...")

# For schema-enabled lakehouses, we need to use the proper namespace
# Format: workspace.lakehouse.schema.table

# Try listing tables in the bronze schema
print("\n1. Listing tables in bronze schema:")
try:
    bronze_tables = spark.sql("SHOW TABLES IN bronze").collect()
    print(f"   Found {len(bronze_tables)} tables in bronze schema")
    for row in bronze_tables:
        print(f"   - {row.tableName}")
except Exception as e:
    print(f"   Error: {str(e)[:200]}...")

# Alternative: List all tables and filter
print("\n2. Listing all tables in current database:")
all_tables = spark.sql("SHOW TABLES").collect()
print(f"   Found {len(all_tables)} total tables")

# Build table mappings
table_mappings = {"bronze": {}, "silver": {}, "gold": {}}

# For schema-enabled lakehouse, tables are referenced as schema.table
print("\n3. Checking bronze schema tables:")
bronze_entity_names = ["agreement", "invoice", "timeentry", "expenseentry", "productitem"]

for entity in bronze_entity_names:
    # Try different table name formats
    table_candidates = [
        f"bronze.bronze_cw_{entity}",  # schema.table format
        f"bronze_cw_{entity}",  # just table name
        f"`bronze`.`bronze_cw_{entity}`",  # quoted format
    ]

    for table_name in table_candidates:
        try:
            count = spark.sql(f"SELECT COUNT(*) FROM {table_name}").collect()[0][0]
            print(f"   ✓ Found {entity}: {table_name} ({count:,} rows)")
            table_mappings["bronze"][entity] = table_name
            table_mappings["silver"][entity] = f"silver.silver_cw_{entity}"
            table_mappings["gold"][entity] = f"gold.gold_cw_{entity}"
            break
        except:
            continue

print(f"\n✅ Discovered {len(table_mappings['bronze'])} bronze tables")
print(f"Table mappings: {table_mappings}")

if table_mappings["bronze"]:
    print("\n🚀 Starting Silver Layer Processing...")
    run_etl_pipeline(
        integrations=["connectwise"], layers=["silver"], config={}, table_mappings=table_mappings
    )
else:
    print("\n❌ No bronze tables found. Check schema configuration.")

StatementMeta(, 89f93da4-344b-49ff-8a91-0fe14404ed3d, 15, Finished, Available, Finished)

🔍 Discovering tables in schema-enabled lakehouse...

1. Listing tables in bronze schema:
   Found 5 tables in bronze schema
   - bronze_cw_agreement
   - bronze_cw_expenseentry
   - bronze_cw_invoice
   - bronze_cw_productitem
   - bronze_cw_timeentry

2. Listing all tables in current database:
   Found 0 total tables

3. Checking bronze schema tables:
   ✓ Found agreement: bronze.bronze_cw_agreement (1,933 rows)
   ✓ Found invoice: bronze.bronze_cw_invoice (37,294 rows)
   ✓ Found timeentry: bronze.bronze_cw_timeentry (535,963 rows)
   ✓ Found expenseentry: bronze.bronze_cw_expenseentry (14,188 rows)


2025-05-28 09:18:47,558 -root - INFO - ✅ Integration 'connectwise' detected and loaded
2025-05-28 09:18:47,559 -root - INFO - ⚠️ Integration 'businesscentral' not available (package not installed)
2025-05-28 09:18:47,561 -root - INFO - ⚠️ Integration 'jira' not available (package not installed)
2025-05-28 09:18:47,562 -root - INFO - Running ETL pipeline for integrations: ['connectwise']
2025-05-28 09:18:47,562 -root - INFO - Processing layers: ['silver']
2025-05-28 09:18:47,563 -root - INFO - Processing integration: connectwise
2025-05-28 09:18:47,563 -root - INFO - Running silver layer for connectwise
2025-05-28 09:18:48,324 -root - INFO - Processing 1933 rows from bronze.bronze_cw_agreement
2025-05-28 09:18:48,365 -root - INFO - Skipping validation - data was validated in Bronze layer
2025-05-28 09:18:48,365 -root - INFO - No column mappings provided, skipping type conversions
2025-05-28 09:18:51,835 -root - INFO - Flattening 28 nested columns: ['type', 'company', 'contact', 'site', 

In [7]:
# Check silver tables
silver_tables = spark.sql("SHOW TABLES LIKE 'silver_cw*'")
silver_tables.show()

# Check row counts and sample data
print("\nSilver Table Summary:")
print("=" * 50)
for row in silver_tables.collect():
    table_name = row.tableName
    df = spark.table(table_name)
    count = df.count()
    print(f"\n{table_name}: {count:,} rows")
    print(f"Columns: {len(df.columns)}")
    print(f"Sample columns: {', '.join(df.columns[:5])}...")

    # Show sample data
    print("\nSample data:")
    df.show(5, truncate=True)

StatementMeta(, 89f93da4-344b-49ff-8a91-0fe14404ed3d, 16, Finished, Available, Finished)

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+


Silver Table Summary:


In [ ]:
print("\n📊 Silver Layer Processing Summary")
print("=" * 50)

# Get all tables and their sizes
all_tables = spark.sql("SHOW TABLES")
table_sizes = {}

for row in all_tables.collect():
    table = row.tableName
    if table.startswith(("bronze_", "silver_")):
        try:
            count = spark.table(table).count()
            table_sizes[table] = count
        except:
            pass

# Group by layer
bronze = {k: v for k, v in table_sizes.items() if k.startswith("bronze_")}
silver = {k: v for k, v in table_sizes.items() if k.startswith("silver_")}

print(f"Bronze Tables: {len(bronze)} tables, {sum(bronze.values()):,} total rows")
print(f"Silver Tables: {len(silver)} tables, {sum(silver.values()):,} total rows")

print("\n✅ Silver processing completed successfully!")
print("\nKey improvements in this version:")
print("- No collect() or row-by-row processing")
print("- Distributed Spark operations throughout")
print("- Scalable to millions of rows")
print("\nNext step: Run Gold layer processing in a separate notebook")

StatementMeta(, 89f93da4-344b-49ff-8a91-0fe14404ed3d, -1, Cancelled, , Cancelled)